In [1]:
print('20232007 김동건')

20232007 김동건


In [2]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [4]:
@deco
def target():
    print("running target()")

target()

running inner()


In [5]:
target

<function __main__.deco.<locals>.inner()>

In [6]:
def logger(func):
    def wrapper(*args, **kwargs):
        print(f"[LOG] {func.__name__} called with args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)
        print(f"[LOG] {func.__name__} returned {result}")
        return result
    return wrapper

@logger
def add(a, b):
    return a + b

In [7]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()

running register(<function f1 at 0x78677de59e40>)
running register(<function f2 at 0x78677de5aac0>)
running main()
registry -> [<function f1 at 0x78677de59e40>, <function f2 at 0x78677de5aac0>]
running f1()
running f2()
running f3()


In [8]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

@promotion
def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

def best_promo(order):
    return max(promo(order) for promo in promos)

In [9]:
def f1(a):
    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

In [10]:
b=6
f1(3)

3
6


In [11]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

f2(3)

3


UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

In [12]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)

3
6


In [13]:
b

9

In [14]:
f3(3)

3
9


In [16]:
b=30
f3(3)

3
30


In [17]:
class Averager():
    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [18]:
avg = Averager()
avg(10)

10.0

In [19]:
avg(11)

10.5

In [20]:
avg(12)

11.0

In [21]:
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)

    return averager

In [22]:
avg = make_averager()
avg(10)

10.0

In [23]:
avg(11)

10.5

In [24]:
avg(12)

11.0

In [26]:
avg.__code__.co_varnames

('new_value', 'total')

In [27]:
avg.__code__.co_freevars

('series',)

In [28]:
avg.__closure__

(<cell at 0x78677e52d900: list object at 0x78677de48540>,)

In [29]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [31]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1
        total += new_value
        return total/count

    return averager

In [32]:
avg = make_averager()
avg(10)

UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

In [33]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count

    return averager

In [35]:
avg = make_averager()
avg(10)

10.0

In [36]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [37]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12318933s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000052s] factorial(1) -> 1
[0.00001472s] factorial(2) -> 2
[0.00002148s] factorial(3) -> 6
[0.00002834s] factorial(4) -> 24
[0.00003636s] factorial(5) -> 120
[0.00004464s] factorial(6) -> 720
6! = 720


In [41]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        arg_lst = []
        name = func.__name__
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [42]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12317061s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000072s] factorial(1) -> 1
[0.00001574s] factorial(2) -> 2
[0.00002289s] factorial(3) -> 6
[0.00003004s] factorial(4) -> 24
[0.00003672s] factorial(5) -> 120
[0.00004601s] factorial(6) -> 720
6! = 720
